In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import DataLoader, Dataset

c:\Users\20520\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df2 = pd.read_csv('data/wikisql/wikisql_schema_train.csv')


In [3]:
questions = df2['question'].tolist()
sql_queries = df2['sql'].tolist()

In [4]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
tokenized_inputs = tokenizer.batch_encode_plus(questions, padding=True, truncation=True, return_tensors='pt')
tokenized_outputs = tokenizer.batch_encode_plus(sql_queries, padding=True, truncation=True, return_tensors='pt')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
class SQLOnlineDataset(Dataset):
    def __init__(self, tokenized_inputs, tokenized_outputs):
        self.input_ids = tokenized_inputs['input_ids']
        self.attention_mask = tokenized_inputs['attention_mask']
        self.labels = tokenized_outputs['input_ids']
        self.decoder_attention_mask = tokenized_outputs['attention_mask']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx],
            'decoder_attention_mask': self.decoder_attention_mask[idx]
        }


In [6]:
train_dataset = SQLOnlineDataset(tokenized_inputs, tokenized_outputs)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [7]:
device = torch.device('cpu')
model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)

In [8]:
optimizer = AdamW(model.parameters(), lr=3e-5)

c:\Users\20520\anaconda3\envs\py310\lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:

num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        decoder_attention_mask = batch['decoder_attention_mask'].to(device)
        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask,
            return_dict=True
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

Epoch: 1, Loss: 2.175539255142212
Epoch: 2, Loss: 1.5234440565109253
Epoch: 3, Loss: 0.6547583937644958
Epoch: 4, Loss: 0.7873156070709229
Epoch: 5, Loss: 0.4106217324733734
Epoch: 6, Loss: 0.27955135703086853
Epoch: 7, Loss: 0.1688154935836792
Epoch: 8, Loss: 0.029750453308224678
Epoch: 9, Loss: 0.1447049379348755
Epoch: 10, Loss: 0.01725548692047596


In [10]:
model.save_pretrained('model/wikisql/sql_t5_model')
tokenizer.save_pretrained('model/wikisql/sql_t5_tokenizer')

('sql_t5_tokenizer\\tokenizer_config.json',
 'sql_t5_tokenizer\\special_tokens_map.json',
 'sql_t5_tokenizer\\spiece.model',
 'sql_t5_tokenizer\\added_tokens.json')

In [11]:
torch.save(model.state_dict(), 'model/wikisql/sql_model.pt')

In [12]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')
model.load_state_dict(torch.load('model/wikisql/sql_model.pt'))
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [13]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
new_question = "What team has more than 49 laps and a grid of 8?"
input_ids = tokenizer.encode(new_question, return_tensors='pt')
outputs = model.generate(input_ids=input_ids, max_length=100, num_beams=5, early_stopping=True)
sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Question: {new_question}")
print(f"Generated SQL query: {sql_query}")

Question: What team has more than 49 laps and a grid of 8?
Generated SQL query: SELECT Team FROM table WHERE Grid = 8
